In [2]:
import pandas as pd
excel_data_globale = pd.read_excel('C:/Users/msi/Downloads/DATA EUROMILLION CHOIX DISCRET (1).xlsx')

In [3]:
# les varaibles exogéne utilisée
liste_columns=['JOUR tirage','E1','E2','pair','impair','pair_E1','sum_b','is_sumpaire','corr']

In [4]:
# Importation des bibliothèques nécessaires
import pandas as pd
import statsmodels.api as sm


# Fonction pour catégoriser la varaible endogéne
def categorize_and_count(row):
    count = 0
    for number in row:
        count = number + count
    if count <=50:
      return 0
    if  50 < count <= 100:
      return 1 
    if  100 < count <= 150:
          return 2 
    if  150 < count <= 200:
          return 3 
    else: 
        return 4

In [5]:
# Application de la fonction aux colonnes B1 à B5 pour créer la variable endogène
excel_data_globale["Endogéne"] = excel_data_globale[['B1', 'B2', 'B3', 'B4', 'B5']].apply( lambda row: categorize_and_count(row), axis=1)

In [7]:
def categorize_étoile(row):
    count = 0
    for number in row:
        count = number + count
    if count <=10:
      return 0 
    else: 
        return 2
    
def categorize_jour(row):
       if row['JOUR tirage'] =="MARDI   ":
           return 0
       else:
          return 1

In [8]:
excel_data_globale['Groupe_E1_E2']= excel_data_globale[['E1', 'E2']].apply( lambda row: categorize_étoile(row), axis=1)
excel_data_globale["JOUR"]= excel_data_globale.apply( lambda row: categorize_jour(row), axis=1)

In [9]:
# Sélection de quelques variables exogènes pour le modèle
Exogéne_varaibles = excel_data_globale[['JOUR','Groupe_E1_E2','pair','impair','pair_E1','sum_b','is_sumpaire','corr','MAX','MIN','produit']]
endogenous_variable = excel_data_globale["Endogéne"]

In [10]:
# Ajout d'une colonne constante aux variables exogènes pour le terme d'interception
X = sm.add_constant(Exogéne_varaibles)

In [11]:
# Création et ajustement du modèle de régression de Poisson avec statsmodels
poisson_model = sm.GLM(endogenous_variable, X, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Affichage du résumé du modèle
poisson_results_summary = poisson_results.summary()

In [12]:
# Affichage du résumé du modèle
poisson_results_summary 

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Endogéne   No. Observations:                 1685
Model:                            GLM   Df Residuals:                     1674
Model Family:                 Poisson   Df Model:                           10
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2211.1
Date:                Thu, 18 Jan 2024   Deviance:                       80.848
Time:                        20:19:53   Pearson chi2:                     78.6
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1599
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0665      0.019     -3.535      0.000      -0.103      -0.030
JOUR             0.0060      0.035      0.169      0.866      -0.063       0.075
Groupe_E1_E2     0.0010      0.018      0.056      0.956      -0.034       0.036
pair            -0.1656      0.048     -3.429      0.001      -0.260      -0.071
impair          -0.1667      0.047     -3.542      0.000      -0.259      -0.074
pair_E1          0.0098      0.034      0.286      0.775      -0.058       0.077
sum_b            0.0111      0.001      9.028      0.000       0.009       0.013
is_sumpaire     -0.0134      0.034     -0.392      0.695      -0.081       0.054
corr             0.0981      0.246      0.398      0.690      -0.385       0.581
MAX              0.0015      0.004      0.370      0.711      -0.006       0.009
MIN              0.0044      0.004      1.002      0.317      -0.004       0.013
produit      -4.317e-09   1.97e-09     -2.192      0.028   -8.18e-09   -4.58e-10
================================================================================
"""

Les résultats de notre modèle GLM Poisson montrent les effets de divers prédicteurs sur la variable dépendante 'Endogéne', avec un total de 1685 observations. Voici les points clés de l'analyse :

Coefficients et Signification Statistique :
'const' a un coefficient négatif (-0.0665) et est statistiquement significatif (p < 0.001), indiquant une influence notable.
'JOUR' et 'Groupe_E1_E2' ne sont pas statistiquement significatifs (p > 0.05), suggérant une influence limitée sur 'Endogéne'.
'pair' (-0.1656) et 'impair' (-0.1667) ont des effets négatifs significatifs.
'sum_b' a un effet positif significatif (0.0111) sur 'Endogéne'.
'pair_E1', 'is_sumpaire', 'corr', 'MAX' et 'MIN' n'ont pas d'effets significatifs.


Modèle dans son Ensemble :
Le modèle présente un bon ajustement avec un log-vraisemblance de -2211.1 et un pseudo R-carré de 0.1599.
La déviance du modèle est de 80.848, indiquant une bonne adéquation entre le modèle et les données observées.


Interprétation Générale :
Les prédicteurs 'pair', 'impair', et 'sum_b' semblent être les plus influents pour prédire 'Endogéne'.

Les autres variables, bien qu'incluses dans le modèle, n'ont pas montré d'influence statistiquement significative.

Ces résultats fournissent des insights précieux sur les relations entre vos prédicteurs et la variable dépendante, aidant à comprendre quels facteurs sont les plus importants dans la prédiction de 'Endogéne'.

In [13]:
# Sélection de quelques variables exogènes pour le modèle
Exogéne_varaibles = excel_data_globale[['pair','impair','sum_b']]

In [14]:
# Ajout d'une colonne constante aux variables exogènes pour le terme d'interception
X = sm.add_constant(Exogéne_varaibles)

In [15]:
# Création et ajustement du modèle de régression de Poisson avec statsmodels
poisson_model = sm.GLM(endogenous_variable, X, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Affichage du résumé du modèle
poisson_results_summary = poisson_results.summary()

In [16]:
poisson_results_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Endogéne   No. Observations:                 1685
Model:                            GLM   Df Residuals:                     1682
Model Family:                 Poisson   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2214.6
Date:                Thu, 18 Jan 2024   Deviance:                       87.763
Time:                        20:20:02   Pearson chi2:                     84.5
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1565
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0435      0.006     -7.220      0.000      -0.055      -0.032
pair          -0.1076      0.017     -6.163      0.000      -0.142      -0.073
impair        -0.1099      0.017     -6.604      0.000      -0.142      -0.077
sum_b          0.0098      0.001     16.818      0.000       0.009       0.011
==============================================================================
"""

Voici une analyse des résultats de votre modèle GLM Poisson :

Performance Générale du Modèle :

Avec 1685 observations, le modèle montre un log-vraisemblance de -2214.6.
Le pseudo R-carré est de 0.1565, indiquant une adéquation raisonnable du modèle avec les données.
La déviance est de 87.763, et le chi2 de Pearson est de 84.5, suggérant que le modèle s'adapte bien aux données observées.




Analyse des Coefficients :

const : Coefficient de -0.0435, significatif (p < 0.0001), indiquant un effet de base notable sur la variable dépendante 'Endogéne'.
pair : Coefficient de -0.1076, également significatif (p < 0.0001). Cela suggère une influence négative sur 'Endogéne' - une augmentation de 'pair' est associée à une diminution de la variable dépendante.
impair : Coefficient de -0.1099, avec une signification similaire (p < 0.0001), indiquant un effet négatif comparable à celui de 'pair'.
sum_b : Coefficient positif de 0.0098, très significatif (p < 0.0001), impliquant une augmentation positive de 'Endogéne' avec l'augmentation de 'sum_b'.   



Implications et Interprétations :

Les variables 'pair' et 'impair' ont des impacts négatifs distincts et significatifs sur la variable 'Endogéne', ce qui indique que ces prédicteurs sont d'importants facteurs influençant la réponse du modèle.

En revanche, 'sum_b' a un effet positif clair, ce qui suggère que des valeurs plus élevées de 'sum_b' sont associées à une augmentation de la variable dépendante.
L'importance de ces prédicteurs est soulignée par leurs p-values très faibles, indiquant une forte signification statistique.


En résumé, ce modèle GLM Poisson met en évidence l'importance et l'influence significative des variables 'pair', 'impair', et 'sum_b' sur la variable dépendante 'Endogéne'.

In [17]:
# Calcul des effets marginaux
marginal_effects = poisson_results.get_margeff()
print(marginal_effects.summary())

         GLM Marginal Effects        
Dep. Variable:               Endogéne
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pair          -0.2184      0.035     -6.248      0.000      -0.287      -0.150
impair        -0.2230      0.033     -6.706      0.000      -0.288      -0.158
sum_b          0.0200      0.001     16.163      0.000       0.018       0.022


In [18]:
import pandas as pd

# Création de la nouvelle observation avec les valeurs fournies
new_observation = {
    'const': [ 1, 1, 1, 1, 1],
    'pair': [0, 1, 2, 3, 4],
    'impair': [4, 3, 2, 1, 0],
    'sum_b': [127.169725, 127.169725, 127.169725, 127.169725, 127.169725]
}

# Conversion en DataFrame
df_new = pd.DataFrame.from_dict(new_observation)
df_new

,const,pair,impair,sum_b
0,1,0,4,127.169725
1,1,1,3,127.169725
2,1,2,2,127.169725
3,1,3,1,127.169725
4,1,4,0,127.169725


In [19]:
# Si df_new n'a pas la colonne 'const', ajoutez-la
if 'const' not in df_new.columns:
    df_new = sm.add_constant(df_new)

# Faire la prédiction en utilisant le modèle
predicted_nombre_point= poisson_results.predict(df_new)
prediction_lambda = list(predicted_nombre_point)

# Création de la nouvelle observation avec les valeurs fournies
new_observation = {
    'const': [1, 1, 1, 1, 1],
    'pair': [0, 1, 2, 3, 4],
    'impair': [4, 3, 2, 1, 0],
    'sum_b': [127.169725, 127.169725, 127.169725, 127.169725, 127.169725],
    'estimation_lambda': prediction_lambda
}

# Conversion en DataFrame
df_new = pd.DataFrame.from_dict(new_observation)
df_new

,const,pair,impair,sum_b,estimation_lambda
0,1,0,4,127.169725,2.159056
1,1,1,3,127.169725,2.163885
2,1,2,2,127.169725,2.168726
3,1,3,1,127.169725,2.173577
4,1,4,0,127.169725,2.178439


In [25]:
y=3 
import numpy as np       
observation = [1, 2, 3, 154 ]   
poisson_results.predict(np.array(observation))

array([2.53085005])

In [26]:
y=3 
import numpy as np       
observation = [1, 3, 2, 158 ]   
poisson_results.predict(np.array(observation))

array([2.63844287])